In [ ]:
# 각각의 함수가 어떤 역할을 하는지 직접 써 주세요.
#__init__:
#__str__:
#__len__:
#__getitem__:

In [ ]:
# 아래에 [] 부분의 함수를 채워주세요.

###############################################################################
# 위의 전체 프로세스를 Dataset 에 모두 정의(데이터 로딩, 전처리, Dataset 생성)
################################################################################
import os
import re
from glob import glob
import tarfile
from PIL import Image

from torch.utils.data import Dataset

# class 정의 및 초기화
class OxfordPetDataset2(Dataset):
    
    def [        ](self, root, split, transform=None):
        """
        Args:
            root(str) - 모든 이미지가 저장된 디렉토리
            split(str) - train/valid/test Dataset중 어떤 dataset을 생성할 지.
            transfomr(callable) - 전처리 callable 객체
        """
        self.root = root  # 파일들이 저장된 root 디렉토리.
        self.split = split  # train / valid / test
        self.transform = transform
        # trainset, validation set 구분 기준 index
        self.train_idx = int(200 * 0.7) # trainset 기준 index
        self.val_idx = self.train_idx + int(200*0.2)
        
        # RGB 이미지 빼고 제거 + file_list 생성
        self.file_list = self._remove_not_rgb()
        self.file_list.sort()  
        # index_to_class, class_to_index 생성
        self.index_to_class, self.class_to_index = self._create_class_index()
        # 파일 경로 목록 생성
        self.split_file_list = self._create_split_file_list(split)
   
# 이미지 수 반환
    def [        ](self):
        return len(self.split_file_list)

# 데이터 불러오기     
    def [           ](self, index):
        path = self.split_file_list[index]
        # x - input
        img = Image.open(path).convert('RGB') 
        img = img.resize((224, 224)) # Transform에서 처리.
        
        if self.transform is not None:
            img = self.transform(img)       
        # y - output
        class_name = re.sub(r"_\d+\.jpg", "", os.path.basename(path))
        class_index = self.class_to_index[class_name]
        return img, class_index
        
# 디버그용 문자열 반환
    def [        ](self):
        return f"OxfordPet Dataset\nSplit: {self.split}\n총데이터수: {self.__len__()}"

# split 별 파일 리스트 생성  
    def [                     ](self, split):
        """
        split(train/valid/test) 별 파일 경로 list 반환
        Args
            split(str) train/valid/test 
        Returns
            list: 파일 경로 List
        """
        split_file_list = []
        cnt = 0
        previous_class = None
        for path in self.file_list:
            file_name = os.path.splitext(os.path.basename(path))[0]  # 디렉토리 빼고 확장자 빼고 파일명만 추출
            class_name = re.sub(r"_\d+", "", file_name)
            if previous_class == class_name:
                cnt += 1
            else:
                cnt = 1

            if split=="train":
                if cnt <= self.train_idx:
                    split_file_list.append(path)
            elif split=="valid":
                if cnt > self.train_idx and cnt <= self.val_idx:
                    split_file_list.append(path)
            elif split=="test":
                if cnt > self.val_idx:
                    split_file_list.append(path)
            else:
                raise Exception(f"split은 train/valid/test 중 하나를 입력하세요.")
            
            previous_class = class_name
            
        return split_file_list

# class index 생성       
    def [            ](self):
        """
        index: class index, class: class_name
        파일명 label을 이용해 index_to_class 리스트, class_to_index dictionary 생성
        
        Returns
            tuple - index_to_class, class_to_index
        """
        class_name_set = set()  # 빈 set. 여기에 파일명들을 저장. -> 중복 제거를 위해서 set사용.
        for file in self.file_list:
            file_name = os.path.basename(file)       # Beagle_32.jpg
            label = re.sub(r'_\d+.jpg', "", file_name)  # Beagle
            class_name_set.add(label)
        index_to_class = list(class_name_set)
        index_to_class.sort()   # [A, B, C, ..]  # 리스트 index: class index, value: class Name
        class_to_index = {value:index for index, value in enumerate(index_to_class)}
        return index_to_class, class_to_index

# RGB가 아닌 이미지 제거
    
    def [          ](self):
        """
        데이터파일에서 RGB 를 제외한 이미지 제거
        """
        file_list = glob(os.path.join(self.root, "*.jpg"))
        for file in file_list:
            # 이미지파일과 연결
            with Image.open(file) as img:
                image_mode = img.mode # "RGB", "L": grayscale, 
                
            if image_mode != "RGB": # RGB 가 아닌 파일은 제거
                os.remove(file)
        
        return glob(os.path.join(self.root, "*.jpg"))  # 파일목록 리턴